 # Day 5 model validation
 模型验证(model validation)其实很简单，就是在选择模型和超参数之后，通过对训练数
据进行学习，对比模型对已知数据的预测值与实际值的差异。

## 1. 错误的验证方法

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X, y)
y_model = model.predict(X)

from sklearn.metrics import accuracy_score
accuracy_score(y, y_model)

1.0

**准确率100%，可能吗？**
<br>根本缺陷:它用同一套数据训练和 评估模型。另外，最近邻模型是一种与距离相关的评估器，只会简单地存储训练数据，然 后把新数据与存储的已知数据进行对比来预测标签。在理想情况下，模型的准确率总是 100%。

## 2. 模型验证正确方法:留出集(holdout set)
先从训练模型 的数据中留出一部分，然后用这部分留出来的数据来检验模型性能。
<br>不过，用留出集进行模型验证有一个缺点，就是模型失去了一部分训练机会。

In [2]:
from sklearn.model_selection import train_test_split
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, train_size=0.5)
model.fit(X1, y1)
y2_model = model.predict(X2)
accuracy_score(y2, y2_model)

/Users/har/py/virtualenv/data_analysis/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


0.9066666666666666

## 3.交叉检验(cross-validation)
交叉检验，也就是做一组拟合，让数据的每个子集既是训练集，又 是验证集.
<br>这种形式的交叉检验被称为两轮交叉检验——将数据集分成两个子集，依次将每个子 集作为验证集。
<br> 当然，可以通过扩展这个概念，在数据中实现更多轮的实验
<br>如下，是一个用用 Scikit-Learn实现的五轮交叉检验

In [3]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

Scikit-Learn 为不同应用场景提供了各种交叉检验方法，都以迭代器(iterator)形式在 model_selection 模块中实现。
<br>例如，我们可能会遇到交叉检验的轮数与样本数相同的极 端情况，也就是说我们每次只有一个样本做测试，其他样本全用于训练。这种交叉检验类 型被称为 LOO(leave-one-out，只留一个)交叉检验:

In [4]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model,  X,  y,  cv=LeaveOneOut())
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

由于我们有 150 个样本，留一法交叉检验会生成 150 轮试验，每次试验的预测结果要么成 功(得分 1.0)，要么失败(得分 0.0)
<br>计算所有试验准确率的均值就可以得到模型的预测 准确性了:

In [5]:
scores.mean()

0.96